In [1]:
%run features_spectral.ipynb
%run load_labels.ipynb

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import keras
from keras.models import load_model
from keras import backend as K
from keras import models
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.utils import to_categorical


In [3]:
num_classes = 2
channels = 32
sfreq = 128

In [4]:
data = PSDh

In [5]:
labels = load_labels()
label = pd.concat([labels['t1_math'], labels['t2_math'],
                  labels['t3_math']]).to_numpy()
label = label.repeat(data.shape[0]//label.shape[0])

# KNN Classifier

In [6]:
K.clear_session()
x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
neigh_clf = KNeighborsClassifier(n_neighbors=2)
neigh_clf.fit(x_train, y_train)
y_pred = neigh_clf.predict(x_test)
y_true = y_test


In [7]:
scores_neigh = neigh_clf.score(x_test, y_test)
precision_neigh = metrics.precision_score(y_true, y_pred, average='macro')
recall_neigh = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_neigh = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_neigh)
print('precision is:', precision_neigh)
print('recall is:', recall_neigh)
print('f1_score is:', f1_score_neigh)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.5575757575757576
precision is: 0.5528042328042329
recall is: 0.5575757575757576
f1_score is: 0.5427317069250607


array([[382, 145],
       [293, 170]])

# SVM Classifier

In [8]:
x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
svm_clf = SVC()
svm_clf.fit(x_train, y_train)
y_pred = svm_clf.predict(x_test)
y_true = y_test


In [9]:
scores_svm = svm_clf.score(x_test, y_test)
precision_svm = metrics.precision_score(y_true, y_pred, average='macro')
recall_svm = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_svm = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_svm)
print('precision is:', precision_svm)
print('recall is:', recall_svm)
print('f1_score is:', f1_score_svm)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.5828282828282828
precision is: 0.5823876624648945
recall is: 0.5828282828282828
f1_score is: 0.5832439288049924


array([[308, 219],
       [194, 269]])

# Neural Network

In [15]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v, num_classes)
x_train, x_test, y_train, y_test = train_test_split(
    data, y_v, test_size=0.33, random_state=42)

In [11]:
x_in = keras.Input(shape=(x_train.shape[1],))
x = Dense(500, kernel_initializer='normal', activation='relu')(x_in)
x = Dense(300, kernel_initializer='normal', activation='relu')(x)
x = Dense(200, kernel_initializer='normal', activation='relu')(x)
x = Dense(50, kernel_initializer='normal', activation='relu')(x)
out_layer = Dense(num_classes, activation='softmax', name='out')(x)
model = keras.Model(x_in, out_layer)
#         model.summary()
# Compile model
#         loss_fn = keras.losses.SparseCategoricalCrossentropy()
#         loss_fn = keras.losses.categorical_crossentropy()

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-10-04 16:11:34.828408: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-04 16:11:34.828537: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
#model.summary()
# Fit the model
model.fit(x_train, y_train, epochs=100, batch_size=128, verbose=0)
# evaluate the model
scores_dnn = model.evaluate(x_test, y_test, verbose=0)

2022-10-04 16:11:34.984823: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-04 16:11:35.209092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-04 16:11:48.829525: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [13]:
y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

31/31 [==============================] - 0s 3ms/step


2022-10-04 16:11:49.142612: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [14]:
precision_dnn = metrics.precision_score(y_true, y_pred, average='macro')
recall_dnn = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_dnn = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_dnn[1])
print('precision is:', precision_dnn)
print('recall is:', recall_dnn)
print('f1_score is:', f1_score_dnn)
metrics.confusion_matrix(y_true, y_pred)

accuracy is: 0.5616161823272705
precision is: 0.5675377407600208
recall is: 0.5616161616161616
f1_score is: 0.5598799276814841


array([[259, 268],
       [166, 297]])